In [1]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [ ]:
from data_retrieval import lipade_groundtruth
from PIL import Image
from transformers import FlavaImageProcessor, FlavaImageModel
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
corpus = "lipade_groundtruth"
rawPath = "../results/raw/" + corpus + "/"
distancePath = "../results/distance/" + corpus + "/"

model = FlavaImageModel.from_pretrained("facebook/flava-full")
feature_extractor = FlavaImageProcessor.from_pretrained("facebook/flava-full")

In [ ]:
x,_,y = lipade_groundtruth.getDataset(mode="similar")
for i in range(len(x)):
    x[i] = Image.open(x[i]).convert('RGB')

In [ ]:
def getEmbedding(img):
    input = feature_extractor(images=[img], return_tensors="pt")
    return model(**input).last_hidden_state.detach().numpy()

embeddings = np.array([getEmbedding(img) for img in tqdm(x)]).reshape(len(x), -1)

distance = 1 - (cosine_similarity(embeddings, embeddings)+1)/2
distance -= np.diag(distance)

In [ ]:
np.save(rawPath + "flava.npy", embeddings)
np.save(distancePath + "flava.npy", distance)